In [ ]:
from yahoo_fin import stock_info as si
from pytickersymbols import PyTickerSymbols


In [ ]:
stock_data = PyTickerSymbols()

uk_stocks = stock_data.get_stocks_by_index('FTSE 100')
XA =stock_data.get_stocks_by_index('S&P 500')
list(XA)

In [ ]:
cx =list(uk_stocks)

In [ ]:
cx[1]

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import ta
from openai import OpenAI
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from config import config
from macros import ticker_service
class StockAnalyzer:
    def __init__(self, ticker, company_name):
        self.ticker = ticker
        self.company_name = company_name
        self.config = config()
        self.stock_dict = ticker_service().all_tickers()
        self.stock_dict = {**self.stock_dict['S&P 500'], **self.stock_dict['FTSE 100']}
     
        self.stock_data = None
        self.company_info = None
        self.openai_client = OpenAI(api_key=self.config.OPENAI_API_KEY)
        self.competitors = self.get_competitors()
  

    def get_competitors(self):
        prompt = f"Given the following list of companies and their ticker symbols:\n\n"
        for ticker, name in self.stock_dict.items():
            prompt += f"{ticker}: {name}\n"
        prompt += f"\nIdentify the top 3 competitors for {self.company_name} ({self.ticker}) from this list. Return the result as a JSON array of ticker symbols."

        try:
            completion = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a financial analyst assistant. Your task is to identify competitors for a given company. Provide your response in the following JSON format: [\"TICKER1\", \"TICKER2\", \"TICKER3\"]"},
                    {"role": "user", "content": prompt}
                ]
            )
            competitors = json.loads(completion.choices[0].message.content)
            return competitors
        except Exception as e:
            print(f"Error in determining competitors: {e}")
            return []

    def get_stock_data(self, period="1y"):
        stock = yf.Ticker(self.ticker)
        self.stock_data = stock.history(period=period)
        return self.stock_data

    def get_company_info(self):
        stock = yf.Ticker(self.ticker)
        self.company_info = stock.info
        return self.company_info

    def get_financial_statements(self):
        stock = yf.Ticker(self.ticker)
        balance_sheet = stock.balance_sheet
        income_stmt = stock.income_stmt
        cash_flow = stock.cash_flow
        return balance_sheet, income_stmt, cash_flow

    def get_news(self, days=7):
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        url = f"https://newsapi.org/v2/everything?q={self.company_name}&from={start_date.date()}&to={end_date.date()}&sortBy=popularity&apiKey={self.config.NEWS_API_KEY}"
        
        response = requests.get(url)
        if response.status_code == 200:
            news_data = response.json()
            return news_data['articles']
        else:
            print(f"Error fetching news: {response.status_code}")
            return None

    def get_earnings_call_transcripts(self):
        url = f"https://seekingalpha.com/symbol/{self.ticker}/earnings/transcripts"
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            transcripts = soup.find_all('a', class_='transcript-link')
            return [{'title': t.text, 'link': 'https://seekingalpha.com' + t['href']} for t in transcripts]
        else:
            print(f"Error fetching earnings call transcripts: {response.status_code}")
            return None

    def calculate_financial_ratios(self):
        if not self.company_info:
            self.get_company_info()
        
        ratios = {
            'P/E Ratio': self.company_info.get('trailingPE', 'N/A'),
            'Forward P/E': self.company_info.get('forwardPE', 'N/A'),
            'PEG Ratio': self.company_info.get('pegRatio', 'N/A'),
            'Price to Book': self.company_info.get('priceToBook', 'N/A'),
            'Debt to Equity': self.company_info.get('debtToEquity', 'N/A'),
            'Return on Equity': self.company_info.get('returnOnEquity', 'N/A'),
            'Return on Assets': self.company_info.get('returnOnAssets', 'N/A'),
            'Profit Margin': self.company_info.get('profitMargins', 'N/A')
        }
        return ratios

    def calculate_technical_indicators(self):
        if self.stock_data is None:
            self.get_stock_data()
        
        data = self.stock_data.copy()
        data['SMA50'] = ta.trend.sma_indicator(data['Close'], window=50)
        data['SMA200'] = ta.trend.sma_indicator(data['Close'], window=200)
        data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
        macd = ta.trend.MACD(data['Close'])
        data['MACD'] = macd.macd()
        data['MACD_Signal'] = macd.macd_signal()
        bollinger = ta.volatility.BollingerBands(data['Close'])
        data['BB_High'] = bollinger.bollinger_hband()
        data['BB_Low'] = bollinger.bollinger_lband()
        return data

    def gpt4_analysis(self, text, task):
        json_formats = {
            "news_sentiment": '''
            {
                "score": float,  // sentiment score between -1 and 1
                "explanation": "string"  // brief explanation of the sentiment
            }
            ''',
            "earnings_call": '''
            {
                "financial_performance": ["string", "string", ...],  // list of key points about financial performance
                "future_outlook": ["string", "string", ...],  // list of key points about future outlook
                "significant_announcements": ["string", "string", ...]  // list of significant announcements
            }
            ''',
            "market_analysis": '''
            {
                "market_conditions": ["string", "string", ...],  // list of key points about current market conditions
                "future_outlook": ["string", "string", ...]  // list of key points about future outlook
            }
            '''
        }

        try:
            completion = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": f"You are a financial analyst assistant. Your task is to {task}. Provide your response in the following JSON format:\n{json_formats[task]}"},
                    {"role": "user", "content": text}
                ]
            )
            return json.loads(completion.choices[0].message.content)
        except Exception as e:
            print(f"Error in GPT-4 analysis: {e}")
            return None

    def analyze_news_sentiment(self, news):
        def process_article(article):
            title = article['title']
            description = article['description']
            content = f"Title: {title}\nDescription: {description}"
            sentiment = self.gpt4_analysis(content, "news_sentiment")
            return sentiment

        with ThreadPoolExecutor(max_workers=5) as executor:
            future_to_article = {executor.submit(process_article, article): article for article in news[:10]}
            sentiments = []
            for future in as_completed(future_to_article):
                sentiment = future.result()
                if sentiment:
                    sentiments.append(sentiment)
        
        return sentiments

    def analyze_earnings_call(self, transcript_link):
        response = requests.get(transcript_link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            transcript_text = soup.find('div', class_='sa-art').get_text()
            analysis = self.gpt4_analysis(transcript_text[:4000], "earnings_call")
            return analysis
        else:
            print(f"Error fetching transcript: {response.status_code}")
            return None

    def get_competitor_data(self):
        data = {}
        for company in [self.ticker] + self.competitors:
            stock = yf.Ticker(company)
            info = stock.info
            data[company] = {
                'Market Cap': info.get('marketCap', 'N/A'),
                'P/E Ratio': info.get('trailingPE', 'N/A'),
                'Revenue Growth': info.get('revenueGrowth', 'N/A'),
                'Profit Margin': info.get('profitMargins', 'N/A')
            }
        return pd.DataFrame(data).T

    def plot_stock_price_with_indicators(self, data):
        plt.figure(figsize=(15, 10))
        
        plt.subplot(2, 1, 1)
        plt.plot(data.index, data['Close'], label='Close Price')
        plt.plot(data.index, data['SMA50'], label='50-day SMA')
        plt.plot(data.index, data['SMA200'], label='200-day SMA')
        plt.fill_between(data.index, data['BB_High'], data['BB_Low'], alpha=0.1)
        plt.title(f'{self.ticker} Stock Price with Technical Indicators')
        plt.legend()
        
        plt.subplot(2, 1, 2)
        plt.plot(data.index, data['RSI'])
        plt.title('Relative Strength Index (RSI)')
        plt.axhline(y=70, color='r', linestyle='--')
        plt.axhline(y=30, color='g', linestyle='--')
        
        plt.tight_layout()
        plt.savefig(f'{self.ticker}_technical_analysis.png')
        plt.close()

    def run_analysis(self):
        print(f"Analyzing {self.company_name} ({self.ticker})")
        print(f"Identified competitors: {', '.join(self.competitors)}")

        # Get stock data and company info
        self.get_stock_data()
        self.get_company_info()

        # Financial ratios and statements
        ratios = self.calculate_financial_ratios()
        balance_sheet, income_stmt, cash_flow = self.get_financial_statements()
        
        print("\nFinancial Ratios:")
        for key, value in ratios.items():
            print(f"{key}: {value}")
        
        # Technical analysis
        stock_data_with_indicators = self.calculate_technical_indicators()
        
        # News sentiment analysis
        news = self.get_news()
        if news:
            print("\nAnalyzing news sentiment...")
            sentiments = self.analyze_news_sentiment(news)
            for sentiment in sentiments:
                print(f"Score: {sentiment['score']}, Explanation: {sentiment['explanation']}")

        # Earnings call analysis
        transcripts = self.get_earnings_call_transcripts()
        if transcripts:
            print("\nAnalyzing latest earnings call...")
            latest_transcript = transcripts[0]
            analysis = self.analyze_earnings_call(latest_transcript['link'])
            if analysis:
                print("Financial Performance:")
                for point in analysis['financial_performance']:
                    print(f"- {point}")
                print("\nFuture Outlook:")
                for point in analysis['future_outlook']:
                    print(f"- {point}")
                print("\nSignificant Announcements:")
                for point in analysis['significant_announcements']:
                    print(f"- {point}")

        # Competitor analysis
        competitor_data = self.get_competitor_data()
        print("\nCompetitor Comparison:")
        print(competitor_data)

        # Overall market analysis
        market_analysis = self.gpt4_analysis(f"Analyze the current market conditions for {self.company_name} and its industry, considering its competitors: {', '.join(self.competitors)}.", "market_analysis")
        if market_analysis:
            print("\nMarket Analysis:")
            print("Current Market Conditions:")
            for point in market_analysis['market_conditions']:
                print(f"- {point}")
            print("\nFuture Outlook:")
            for point in market_analysis['future_outlook']:
                print(f"- {point}")

        # Plot technical indicators
        self.plot_stock_price_with_indicators(stock_data_with_indicators)
        print(f"\nTechnical analysis plot saved as '{self.ticker}_technical_analysis.png'")

# Usage
if __name__ == "__main__":
 
     
    analyzer = StockAnalyzer("AAPL", "Apple Inc.")
    analyzer.run_analysis()

INFO:macros:S&P 500 tickers: 499
INFO:macros:FTSE 100 tickers: 102
INFO:macros:All tickers have been saved to 'all_tickers.json'
INFO:macros:S&P 500 tickers: 499
INFO:macros:FTSE 100 tickers: 102
INFO:macros:All tickers have been saved to 'all_tickers.json'
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Analyzing Apple Inc. (AAPL)
Identified competitors: MSFT, GOOGL, AMZN

Financial Ratios:
P/E Ratio: 33.917683
Forward P/E: 29.745989
PEG Ratio: 3.0
Price to Book: 50.7759
Debt to Equity: 151.862
Return on Equity: 1.60583
Return on Assets: 0.22612
Profit Margin: 0.26441

Analyzing news sentiment...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Score: 0.6, Explanation: The sentiment is primarily positive as it highlights a price reduction in Apple products, which is usually received positively by consumers.
Score: -0.1, Explanation: The news shows a neutral to slightly negative sentiment. Although it mentions a potentially significant political event, it also describes a level of disinterest from some sections of the business world, which could be viewed as negative.
Score: 1, Explanation: The news is extremely positive as it talks about Apple's innovative advancements in AI technology with its new products, which gives clear indications of growth and progress.
Score: 0.7, Explanation: The news suggests a positive sentiment as Apple is planning to decrease the manufacturing costs of its future Apple Vision Pro headset. This will likely make the product more accessible to customers due to a potential price reduction, further benefitting the company's sales.
Score: 0.2, Explanation: The news is slightly positive. Although it st

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Market Analysis:
Current Market Conditions:
- Apple Inc. stays dominant primarily in smartphone and tech markets, showing consistent product demand and robust financials.
- Microsoft (MSFT) actively expands within the cloud-computing and professional software, creating significant competition in the tech sector.
- Google (GOOGL) is a market leader in digital advertising, cloud services, and develops the Android operating system, Apple's main competitor in the smartphone sector.
- Amazon (AMZN) maintains a stronghold in the e-commerce and cloud service space, while also pursuing other avenues which increased competition within the tech sector.
- Global semiconductor shortage is affecting the whole tech industry, likely impacting Apple and its competitors.
- Increased scrutiny from global regulators on tech giants could bring headwinds in the future.

Future Outlook:
- Apple's continued investment in R&D suggests potential for innovation and new product lines.
- Microsoft's success in c